In [1]:
# import keras as ks
import numpy as np
import tensorflow as tf
import pickle

In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
new_saver = tf.train.import_meta_graph('seq2seq_eng2hin_50000-90.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))

graph=tf.get_default_graph()


INFO:tensorflow:Restoring parameters from ./seq2seq_eng2hin_50000-90


In [3]:
file1=open("./mappings_eng.pickle","rb")
file2=open("./mappings_hin.pickle","rb")

mapping_eng,demapping_eng=pickle.load(file1)
mapping_hin,demapping_hin=pickle.load(file2)

In [4]:
names=[tensor.name for tensor in tf.get_default_graph().as_graph_def().node]
names[:5]

['init',
 'save/RestoreV2/shape_and_slices',
 'save/RestoreV2/tensor_names',
 'save/SaveV2/shape_and_slices',
 'save/SaveV2/tensor_names']

In [5]:
X=graph.get_tensor_by_name("Placeholder:0")
Y=graph.get_tensor_by_name("Placeholder_1:0")
logits=graph.get_tensor_by_name("optimizer/fully_connected/BiasAdd:0")

In [6]:
max_e=10
max_h=10
def get_reply(text):
    tokens=text.split()
    data=[]
    data.append(mapping_eng["startseq"])
    m=2
    for i in tokens:
        if m >= 8:
           break
        m+=1
        if i.lower() not in mapping_eng.keys():
          data.append(mapping_eng["UNK"])
        else:
          data.append(mapping_eng[i.lower()])
        
    data.append(mapping_eng["endseq"])
    data_x=np.array([0]*(max_e-len(data))+data)
    data_y=[]
    data_y=np.array([mapping_hin["startseq"]]+[0]*(max_h-len(data_y)-1))
    last_t=""
    
    k=0
    reply=[]
    
    while(last_t!="endseq" and k<max_h-1):
#         print(data_x.shape,data_y.shape)
        reply.append(sess.run(logits,feed_dict={X:[data_x],Y:[data_y]}).argmax(axis=-1)[0,k])
#         print(reply)
        data_y[k+1]=reply[k]
#         print(data_y)
        if(reply[k]==0):
            break
        last_t=demapping_hin[reply[k]]
        k+=1
    text=""
#     print(reply)
    for i in reply:
#       print(i)
      if i==0 or i==mapping_hin["endseq"]:
          break
      text=text+" "+demapping_hin[i]
    return text

In [7]:
get_reply("leave the game")

' खेल छोड़ें'

In [8]:
get_reply("thanks for help")

' मदद के लिए धन्यवाद'

In [9]:
get_reply("how are you")

' कैसे आप कैसे मिलता है'

In [8]:
get_reply("thanks for the invite")

' UNK के लिए धन्यवाद'

In [9]:
get_reply("i love ")

' आप'